In [ ]:
import requests
import json
import pandas as pd
import time
from google.colab import files

# 설정
yourkey = '38c996c10f27e8682b1dbd414f37f46f'
year = '2024'
cur_page = 1
movie_codes = []
all_movie_info = []

# 영화 목록 조회
while True:
    print(f"📦 {cur_page}페이지 영화 목록 요청 중...")

    url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json'
    params = {
        'key': yourkey,
        'openStartDt': year,
        'curPage': cur_page,
        'itemPerPage': 100
    }

    res = requests.get(url, params=params)
    data = res.json()

    if 'movieListResult' not in data:
        print("❌ 오류 또는 데이터 없음")
        break

    movie_list = data['movieListResult']['movieList']
    if not movie_list:
        print("✅ 모든 영화 리스트 조회 완료")
        break

    for m in movie_list:
        movie_codes.append(m['movieCd'])

    cur_page += 1
    time.sleep(0.2)  # 호출 제한 대비

# 상세정보 수집
print(f"총 {len(movie_codes)}개의 영화 상세정보 수집 시작...")

for movieCd in movie_codes:
    detail_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={yourkey}&movieCd={movieCd}'
    res = requests.get(detail_url)
    data = res.json()

    if 'movieInfoResult' not in data:
        continue

    info = data['movieInfoResult']['movieInfo']
    info_flat = {
        'movieCd': info.get('movieCd'),
        'movieNm': info.get('movieNm'),
        'movieNmEn': info.get('movieNmEn'),
        'showTm': info.get('showTm'),
        'prdtYear': info.get('prdtYear'),
        'openDt': info.get('openDt'),
        'genres': ', '.join([g['genreNm'] for g in info.get('genres', [])]),
        'directors': ', '.join([d['peopleNm'] for d in info.get('directors', [])]),
        'actors': ', '.join([a['peopleNm'] for a in info.get('actors', [])][:3])
    }
    all_movie_info.append(info_flat)
    time.sleep(0.1)

# 저장 및 다운로드
filename = f"{year}_movie_detail_info.csv"
df = pd.DataFrame(all_movie_info)
df.to_csv(filename, encoding='utf-8-sig', index=False)
print(f"✅ 저장 완료: {filename}")


files.download(filename)


📦 1페이지 영화 목록 요청 중...
📦 2페이지 영화 목록 요청 중...
📦 3페이지 영화 목록 요청 중...
📦 4페이지 영화 목록 요청 중...
📦 5페이지 영화 목록 요청 중...
📦 6페이지 영화 목록 요청 중...
📦 7페이지 영화 목록 요청 중...
📦 8페이지 영화 목록 요청 중...
📦 9페이지 영화 목록 요청 중...
📦 10페이지 영화 목록 요청 중...
📦 11페이지 영화 목록 요청 중...
📦 12페이지 영화 목록 요청 중...
📦 13페이지 영화 목록 요청 중...
📦 14페이지 영화 목록 요청 중...
📦 15페이지 영화 목록 요청 중...
📦 16페이지 영화 목록 요청 중...
📦 17페이지 영화 목록 요청 중...
📦 18페이지 영화 목록 요청 중...
📦 19페이지 영화 목록 요청 중...
📦 20페이지 영화 목록 요청 중...
✅ 모든 영화 리스트 조회 완료
총 1888개의 영화 상세정보 수집 시작...
✅ 저장 완료: 2024_movie_detail_info.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import requests
import time

# TMDB API 정보
API_KEY = "f5d5730b235892f8ee654e71309d89a9"
SEARCH_URL = "https://api.themoviedb.org/3/search/movie"
POSTER_BASE_URL = "https://image.tmdb.org/t/p/w500"

# 1. 원본 CSV 파일 불러오기
df = pd.read_csv("/content/movie_data/2024_movie_detail_info.csv")

# 2. 포스터 URL 컬럼 추가 (초기값: None)
df["poster_url"] = None

# 3. 영화명(영문)과 연도로 검색해서 포스터 URL 추가
for idx, row in df.iterrows():
    title = row["movieNmEn"]
    if pd.isna(title) or title.lower() == "known":
        continue

    year = int(row["prdtYear"]) if not pd.isna(row["prdtYear"]) else None

    params = {
        "api_key": API_KEY,
        "query": title,
        "year": year
    }

    try:
        response = requests.get(SEARCH_URL, params=params)
        data = response.json()
        results = data.get("results")

        if results:
            poster_path = results[0].get("poster_path")
            if poster_path:
                df.at[idx, "poster_url"] = POSTER_BASE_URL + poster_path
            else:
                df.at[idx, "poster_url"] = pd.NA
        else:
            df.at[idx, "poster_url"] = pd.NA

    except Exception as e:
        df.at[idx, "poster_url"] = f"오류: {e}"

    time.sleep(0.2)  # 과도한 호출 방지

# 4. 결과 저장 (덮어쓰기 or 새로운 파일명)
df.to_csv("2024_movie_detail_info_with_posters.csv", index=False)
print("포스터 URL 추가 완료!")

포스터 URL 추가 완료!


In [ ]:
movie_df = pd.read_csv('/content/2024_movie_detail_info_with_posters.csv')
movie_df

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url
0,20254602,씨너스: 죄인들,Sinners,137.0,2025.0,20250528,액션,라이언 쿠글러,"마이클 B. 조던, 헤일리 스테인펠드",https://image.tmdb.org/t/p/w500/jYfMTSiFFK7ffb...
1,20247176,드래곤 길들이기,How to Train Your Dragon,125.0,2025.0,20250606,"액션, 어드벤처, 판타지",딘 데블로이스,"메이슨 테임즈, 제라드 버틀러, 니코 파커",https://image.tmdb.org/t/p/w500/q5pXRYTycaeW6d...
2,20254174,366일,366 Days,123.0,2024.0,20250611,멜로/로맨스,신조 타케히코,"아카소 에이지, 카미시라이시 모카, 나카지마 유토",NaN
3,20253236,하보우만의 약속,known,87.0,2025.0,20250416,다큐멘터리,이장호,NaN,NaN
4,20252735,여성국극 끊어질듯 이어지고 사라질듯 영원하다,Women’s Gukgeuk: Enduring on the Edge of Time,104.0,2024.0,20250319,다큐멘터리,유수연,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1883,20236895,흥건한 며느리와 대물 시아버지,NaN,73.0,2023.0,20240103,성인물(에로),NaN,NaN,NaN
1884,20235924,클레오의 세계,Ama Gloria,84.0,2023.0,20240103,드라마,마리 아마추켈리,"루이스 모루아-팡자니, 일사 모레노 제고",https://image.tmdb.org/t/p/w500/vfi3h3ZemjWCya...
1885,20236896,장모의 혀놀림에 정신 못차리는 사위,NaN,73.0,2023.0,20240103,성인물(에로),NaN,NaN,NaN
1886,20230984,간신의 피,The Blood of Desire,102.0,2023.0,20240103,액션,권하,"송길호, 기주봉, 현리원",NaN


In [ ]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888 entries, 0 to 1887
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieCd     1888 non-null   object 
 1   movieNm     1888 non-null   object 
 2   movieNmEn   1124 non-null   object 
 3   showTm      1881 non-null   float64
 4   prdtYear    1880 non-null   float64
 5   openDt      1888 non-null   int64  
 6   genres      1882 non-null   object 
 7   directors   976 non-null    object 
 8   actors      652 non-null    object 
 9   poster_url  674 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 147.6+ KB


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies_df = pd.read_csv('/content/2024_movie_detail_info_with_posters_utf8.csv')
movies_df

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url
0,20254602,씨너스: 죄인들,Sinners,137.0,2025.0,20250528,액션,라이언 쿠글러,"마이클 B. 조던, 헤일리 스테인펠드",https://image.tmdb.org/t/p/w500/jYfMTSiFFK7ffb...
1,20247176,드래곤 길들이기,How to Train Your Dragon,125.0,2025.0,20250606,"액션, 어드벤처, 판타지",딘 데블로이스,"메이슨 테임즈, 제라드 버틀러, 니코 파커",https://image.tmdb.org/t/p/w500/q5pXRYTycaeW6d...
2,20254174,366일,366 Days,123.0,2024.0,20250611,멜로/로맨스,신조 타케히코,"아카소 에이지, 카미시라이시 모카, 나카지마 유토",NaN
3,20253236,하보우만의 약속,known,87.0,2025.0,20250416,다큐멘터리,이장호,NaN,NaN
4,20252735,여성국극 끊어질듯 이어지고 사라질듯 영원하다,Women’s Gukgeuk: Enduring on the Edge of Time,104.0,2024.0,20250319,다큐멘터리,유수연,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1883,20236895,흥건한 며느리와 대물 시아버지,NaN,73.0,2023.0,20240103,성인물(에로),NaN,NaN,NaN
1884,20235924,클레오의 세계,Ama Gloria,84.0,2023.0,20240103,드라마,마리 아마추켈리,"루이스 모루아-팡자니, 일사 모레노 제고",https://image.tmdb.org/t/p/w500/vfi3h3ZemjWCya...
1885,20236896,장모의 혀놀림에 정신 못차리는 사위,NaN,73.0,2023.0,20240103,성인물(에로),NaN,NaN,NaN
1886,20230984,간신의 피,The Blood of Desire,102.0,2023.0,20240103,액션,권하,"송길호, 기주봉, 현리원",NaN


In [ ]:
# 장르가 성인물(에로)인 행 삭제
idx = movies_df[movies_df['genres'] == '성인물(에로)'].index
idx

Index([  35,   72,   93,   99,  108,  111,  114,  117,  119,  126,
       ...
       1862, 1864, 1866, 1868, 1877, 1878, 1880, 1882, 1883, 1885],
      dtype='int64', length=378)

In [ ]:
movies_df.drop(idx, inplace = True)

In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1510 entries, 0 to 1887
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieCd     1510 non-null   object 
 1   movieNm     1510 non-null   object 
 2   movieNmEn   1113 non-null   object 
 3   showTm      1507 non-null   float64
 4   prdtYear    1508 non-null   float64
 5   openDt      1510 non-null   int64  
 6   genres      1504 non-null   object 
 7   directors   933 non-null    object 
 8   actors      652 non-null    object 
 9   poster_url  674 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 129.8+ KB


In [ ]:
# 포스터 url이 없는 행 삭제
idx2 = movies_df[movies_df['poster_url'].isnull()].index
idx2

Index([   2,    3,    4,    7,    8,    9,   13,   15,   16,   20,
       ...
       1852, 1853, 1854, 1855, 1870, 1875, 1876, 1879, 1881, 1886],
      dtype='int64', length=836)

In [ ]:
movies_df.drop(idx2, inplace = True)

In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 674 entries, 0 to 1887
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieCd     674 non-null    object 
 1   movieNm     674 non-null    object 
 2   movieNmEn   674 non-null    object 
 3   showTm      672 non-null    float64
 4   prdtYear    674 non-null    float64
 5   openDt      674 non-null    int64  
 6   genres      674 non-null    object 
 7   directors   556 non-null    object 
 8   actors      508 non-null    object 
 9   poster_url  674 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 57.9+ KB


In [ ]:
# 배우가 비워져 있는 행 삭제
idx3 = movies_df[movies_df['actors'].isnull()].index
idx3

Index([], dtype='int64')

In [ ]:
movies_df.drop(idx3, inplace = True)
movies_df

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url
0,20254602,씨너스: 죄인들,Sinners,137.0,2025.0,20250528,액션,라이언 쿠글러,"마이클 B. 조던, 헤일리 스테인펠드",https://image.tmdb.org/t/p/w500/jYfMTSiFFK7ffb...
1,20247176,드래곤 길들이기,How to Train Your Dragon,125.0,2025.0,20250606,"액션, 어드벤처, 판타지",딘 데블로이스,"메이슨 테임즈, 제라드 버틀러, 니코 파커",https://image.tmdb.org/t/p/w500/q5pXRYTycaeW6d...
6,20212932,탈주,Escape,94.0,2024.0,20240703,액션,이종필,"이제훈, 구교환, 홍사빈",https://image.tmdb.org/t/p/w500/lWKtVX5oEgcLbM...
10,20247590,릴로 & 스티치,Lilo & Stitch,107.0,2025.0,20250521,애니메이션,딘 플라이셔 캠프,크리스 샌더스,https://image.tmdb.org/t/p/w500/3bN675X0K2E5Qi...
11,20253581,해피해피,Happy Happy,91.0,2025.0,20250529,드라마,이수성,"길은혜, 황유주, 신지훈",https://image.tmdb.org/t/p/w500/oHqmSc7J9w4Zho...
...,...,...,...,...,...,...,...,...,...,...
1871,20235583,인투 더 월드,Migration,91.0,2023.0,20240110,애니메이션,벤자민 레너,"엘리자베스 뱅크스, 캐롤 케인, 아콰피나",https://image.tmdb.org/t/p/w500/dpXxBDi0ut5svj...
1872,20236014,메이킹 다큐멘터리 `스즈메의 문단속`을 따라가다,"Making Documentary of ""Suzume""",78.0,2023.0,20240110,다큐멘터리,마츠모토 소,"신카이 마코토, 카와무라 겐키",https://image.tmdb.org/t/p/w500/ynCt7v5VVKSwaA...
1874,20235834,위시,WISH,94.0,2023.0,20240103,애니메이션,"폰 비라선손, 크리스 벅","아리아나 데보스, 크리스 파인, 알란 터딕",https://image.tmdb.org/t/p/w500/m9ckCVKU9vW3Dx...
1884,20235924,클레오의 세계,Ama Gloria,84.0,2023.0,20240103,드라마,마리 아마추켈리,"루이스 모루아-팡자니, 일사 모레노 제고",https://image.tmdb.org/t/p/w500/vfi3h3ZemjWCya...


In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 508 entries, 0 to 1887
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieCd     508 non-null    object 
 1   movieNm     508 non-null    object 
 2   movieNmEn   508 non-null    object 
 3   showTm      506 non-null    float64
 4   prdtYear    508 non-null    float64
 5   openDt      508 non-null    int64  
 6   genres      508 non-null    object 
 7   directors   465 non-null    object 
 8   actors      508 non-null    object 
 9   poster_url  508 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 43.7+ KB


In [ ]:
# directors 비워져 있는 행 삭제
idx4 = movies_df[movies_df['directors'].isnull()].index
idx4

Index([], dtype='int64')

In [ ]:
movies_df.drop(idx4, inplace = True)

In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 465 entries, 0 to 1887
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieCd     465 non-null    object 
 1   movieNm     465 non-null    object 
 2   movieNmEn   465 non-null    object 
 3   showTm      463 non-null    float64
 4   prdtYear    465 non-null    float64
 5   openDt      465 non-null    int64  
 6   genres      465 non-null    object 
 7   directors   465 non-null    object 
 8   actors      465 non-null    object 
 9   poster_url  465 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 40.0+ KB


In [ ]:
movies_df.to_csv('/content/data_cleaning_movies.csv', encoding='utf-8-sig', index = False)

In [ ]:
file_path = "/content/data_cleaning_movies.csv"
df = pd.read_csv(file_path)

# 'openDt' 컬럼에서 2024로 시작하는 행만 필터링
filtered_df = df[df['openDt'].astype(str).str.startswith("2024")]

In [ ]:
filtered_df.sort_values(by = 'openDt', ascending = False)

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url
155,20244850,시빌 워: 분열의 시대,Civil War,109.0,2024.0,20241231,"액션, 전쟁, 드라마",알렉스 가랜드,"커스틴 던스트, 케일리 스패니, 와그너 모라",https://image.tmdb.org/t/p/w500/sh7Rg8Er3tFcN9...
116,20190300,보고타: 마지막 기회의 땅,Bogota: City of the Lost,106.0,2024.0,20241231,"범죄, 드라마",김성제,"송중기, 이희준, 권해효",https://image.tmdb.org/t/p/w500/p1WPGGPVHHVJwT...
154,20246201,니코: 오로라 원정대의 모험,Niko: Beyond the Northern Lights,85.0,2024.0,20241225,애니메이션,"캐리 주스넌, 요르겐 레르담","옹성우, 김지은, 박예린",https://image.tmdb.org/t/p/w500/qqt2atvzPJJBvt...
157,20204422,오랜만이다,How Have You Been,92.0,2022.0,20241225,드라마,이은정,"방민아, 이가섭",https://image.tmdb.org/t/p/w500/q6nq8UfyRGjhtc...
141,20246777,더 폴: 디렉터스 컷,The Fall,119.0,2024.0,20241225,"드라마, 판타지",타셈 싱,"리 페이스, 카틴카 언타루",https://image.tmdb.org/t/p/w500/g3RKh7Gp2lDUnX...
...,...,...,...,...,...,...,...,...,...,...
460,20235583,인투 더 월드,Migration,91.0,2023.0,20240110,애니메이션,벤자민 레너,"엘리자베스 뱅크스, 캐롤 케인, 아콰피나",https://image.tmdb.org/t/p/w500/dpXxBDi0ut5svj...
461,20236014,메이킹 다큐멘터리 `스즈메의 문단속`을 따라가다,"Making Documentary of ""Suzume""",78.0,2023.0,20240110,다큐멘터리,마츠모토 소,"신카이 마코토, 카와무라 겐키",https://image.tmdb.org/t/p/w500/ynCt7v5VVKSwaA...
462,20235834,위시,WISH,94.0,2023.0,20240103,애니메이션,"폰 비라선손, 크리스 벅","아리아나 데보스, 크리스 파인, 알란 터딕",https://image.tmdb.org/t/p/w500/m9ckCVKU9vW3Dx...
463,20235924,클레오의 세계,Ama Gloria,84.0,2023.0,20240103,드라마,마리 아마추켈리,"루이스 모루아-팡자니, 일사 모레노 제고",https://image.tmdb.org/t/p/w500/vfi3h3ZemjWCya...


In [ ]:
# showTm이 100분 이하인 행들 삭제
filtered_df = filtered_df[filtered_df['showTm'] > 100]
filtered_df

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url
30,20228796,하얼빈,HARBIN,113.0,2024.0,20241224,드라마,우민호,"현빈, 박정민, 조우진",https://image.tmdb.org/t/p/w500/oNA9eRaFCLlFKR...
49,20227570,울산의 별,Star of Ulsan,117.0,2022.0,20240124,드라마,정기혁,"김금순, 최우빈, 변중희",https://image.tmdb.org/t/p/w500/7EnBkQpZKORRi8...
55,20200142,소방관,FIREFIGHTERS,106.0,2024.0,20241204,드라마,곽경택,"주원, 곽도원, 유재명",https://image.tmdb.org/t/p/w500/8claugsQFr5SAx...
64,20234686,그 여름날의 거짓말,That Summer′s Lie,138.0,2024.0,20240828,"멜로/로맨스, 미스터리",손현록,"박서윤, 최민재, 유병용",https://image.tmdb.org/t/p/w500/7TiQBVFFVsjLxh...
73,20204271,행복의 나라,Land of Happiness,124.0,2024.0,20240814,드라마,추창민,"조정석, 이선균, 유재명",https://image.tmdb.org/t/p/w500/c5xAVbSdJYNVjA...
...,...,...,...,...,...,...,...,...,...,...
451,20236759,소녀는 졸업하지 않는다,"Sayonara, Girls",120.0,2023.0,20240124,멜로/로맨스,나카가와 슌,"카와이 유미, 오노 리나, 코미야마 리나",https://image.tmdb.org/t/p/w500/k41x8dE8spurWv...
453,20235854,덤 머니,Dumb Money,105.0,2023.0,20240117,"코미디, 드라마",크레이그 질레스피,"폴 다노, 세스 로건, 아메리카 페레라",https://image.tmdb.org/t/p/w500/e9u7luSxFKOZgP...
456,20235073,라이즈,Rise,117.0,2022.0,20240117,"드라마, 코미디",세드릭 클라피쉬,"마리옹 바르보, 메디 바키, 호페쉬 쉑터",https://image.tmdb.org/t/p/w500/slkVQhehnP3h40...
459,20209574,히즈,his,126.0,2020.0,20240110,드라마,이마이즈미 리키야,"미야자와 히오, 후지와라 키세츠",https://image.tmdb.org/t/p/w500/ePBDOSLQ26voWs...


In [ ]:
filtered_df.to_csv('/content/2024_movies.csv', encoding='utf-8-sig', index = False)

# TMDB에서 keyword, vote_average, vote_count 가져오기

In [ ]:
import pandas as pd
import requests
import time

# CSV 불러오기
df = pd.read_csv("/content/2024_movies.csv")

# TMDB API
API_KEY = "f5d5730b235892f8ee654e71309d89a9"
SEARCH_URL = "https://api.themoviedb.org/3/search/movie"
DETAIL_URL = "https://api.themoviedb.org/3/movie/{}"
KEYWORDS_URL = "https://api.themoviedb.org/3/movie/{}/keywords"

# 결과 저장용 리스트
results = []

# 전체 영화 반복
for idx, row in df.iterrows():
    title = row["movieNm"]
    year = int(row["prdtYear"])

    print(f"[{idx+1}/{len(df)}] 검색 중: {title} ({year})")

    #  영화 검색
    search_params = {
        "api_key": API_KEY,
        "query": title,
        "year": year,
        "language": "ko-KR"
    }
    try:
        search_response = requests.get(SEARCH_URL, params=search_params).json()
        search_results = search_response.get("results", [])

        if not search_results:
            print(f"  → 검색 실패: {title}")
            continue

        movie_id = search_results[0]["id"]

        #  상세 정보
        detail_response = requests.get(DETAIL_URL.format(movie_id), params={"api_key": API_KEY, "language": "ko-KR"}).json()
        vote_average = detail_response.get("vote_average")
        vote_count = detail_response.get("vote_count")

        #  키워드
        keyword_response = requests.get(KEYWORDS_URL.format(movie_id), params={"api_key": API_KEY}).json()
        keywords = [kw["name"] for kw in keyword_response.get("keywords", [])]

        #  결과 저장
        results.append({
            "title": title,
            "keywords": ", ".join(keywords),
            "vote_average": vote_average,
            "vote_count": vote_count
        })

    except Exception as e:
        print(f"  → 에러 발생: {e}")

    # TMDB API rate limit 회피
    time.sleep(0.3)

# 결과 저장
df_result = pd.DataFrame(results)
df_result.to_csv("tmdb_movie_data_full.csv", index=False, encoding="utf-8-sig")

print("완료! 결과가 'tmdb_movie_data_full.csv'에 저장되었습니다.")

[1/206] 검색 중: 하얼빈 (2024)
[2/206] 검색 중: 울산의 별 (2022)
[3/206] 검색 중: 소방관 (2024)
[4/206] 검색 중: 그 여름날의 거짓말 (2024)
[5/206] 검색 중: 행복의 나라 (2024)
[6/206] 검색 중: 딸에 대하여 (2023)
[7/206] 검색 중: 보고타: 마지막 기회의 땅 (2024)
[8/206] 검색 중: 더 킬러스 (2023)
[9/206] 검색 중: 대가족 (2023)
  → 검색 실패: 대가족
[10/206] 검색 중: 여름이 끝날 무렵의 라트라비아타 (2023)
[11/206] 검색 중: 아침바다 갈매기는 (2024)
[12/206] 검색 중: 소풍 (2024)
[13/206] 검색 중: 더 폴: 디렉터스 컷 (2024)
[14/206] 검색 중: 1승 (2021)
  → 검색 실패: 1승
[15/206] 검색 중: 청설 (2024)
[16/206] 검색 중: 극장판 짱구는 못말려: 우리들의 공룡일기 (2024)
[17/206] 검색 중: 아노라 (2024)
[18/206] 검색 중: 시빌 워: 분열의 시대 (2024)
[19/206] 검색 중: 아마존 활명수 (2024)
[20/206] 검색 중: 서브스턴스 (2024)
[21/206] 검색 중: 무파사: 라이온 킹 (2024)
[22/206] 검색 중: 섬·망(望) (2024)
  → 검색 실패: 섬·망(望)
[23/206] 검색 중: 블랙 나이트 퍼레이드 (2022)
[24/206] 검색 중: 에픽하이 20 더 무비 (2024)
[25/206] 검색 중: 더 크로우 (2024)
[26/206] 검색 중: 오늘부터 1일 (2024)
[27/206] 검색 중: 히든페이스 (2024)
[28/206] 검색 중: 카인의 도시 (2024)
[29/206] 검색 중: 모스크바 미션 (2023)
[30/206] 검색 중: 극장판 미스터리라 하지 말지어다 (2023)
[31/206] 검색 중: 위키드 (2024)
[32/206] 검색 중

In [ ]:
tmdb_df = pd.read_csv('/content/tmdb_movie_data_full.csv')
tmdb_df

,title,keywords,vote_average,vote_count
0,하얼빈,"espionage, spy, independence movement, biograp...",7.733,15
1,울산의 별,NaN,5.000,1
2,소방관,"based on true story, firefighter",7.750,8
3,그 여름날의 거짓말,NaN,0.000,0
4,행복의 나라,"court, 1970s, lawyer, korea president",6.800,10
...,...,...,...,...
185,소녀는 졸업하지 않는다,"high school, farewell, japan, based on novel o...",9.300,7
186,덤 머니,"massachusetts, manipulation of the media, base...",6.700,641
187,라이즈,"sports, biography, basketball, national basket...",7.607,276
188,히즈,"rural area, second chance, lgbt, gay theme, in...",7.200,24


In [ ]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         190 non-null    object 
 1   keywords      144 non-null    object 
 2   vote_average  190 non-null    float64
 3   vote_count    190 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ KB


In [ ]:
# keywords 비어있는 행 삭제
idx5 = tmdb_df[tmdb_df['keywords'].isnull()].index
idx5

Index([  1,   3,   8,  10,  20,  22,  24,  25,  34,  36,  37,  41,  49,  50,
        52,  56,  57,  68,  72,  76,  77,  81,  83,  93,  97,  99, 104, 106,
       114, 118, 120, 121, 126, 128, 133, 138, 149, 150, 151, 162, 168, 169,
       171, 173, 178, 179],
      dtype='int64')

In [ ]:
tmdb_df.drop(idx5, inplace = True)

KeyError: '[1, 3, 8, 10, 20, 22, 24, 25, 34, 36, 37, 41, 49, 50, 52, 56, 57, 68, 72, 76, 77, 81, 83, 93, 97, 99, 104, 106, 114, 118, 120, 121, 126, 128, 133, 138, 149, 150, 151, 162, 168, 169, 171, 173, 178, 179] not found in axis'

In [ ]:
tmdb_df

,title,keywords,vote_average,vote_count
0,하얼빈,"espionage, spy, independence movement, biograp...",7.733,15
2,소방관,"based on true story, firefighter",7.750,8
4,행복의 나라,"court, 1970s, lawyer, korea president",6.800,10
5,딸에 대하여,"based on novel or book, mother daughter relati...",6.500,3
6,보고타: 마지막 기회의 땅,"bogotá, 1990s, colombia, bogota, colombia, sus...",6.982,85
...,...,...,...,...
185,소녀는 졸업하지 않는다,"high school, farewell, japan, based on novel o...",9.300,7
186,덤 머니,"massachusetts, manipulation of the media, base...",6.700,641
187,라이즈,"sports, biography, basketball, national basket...",7.607,276
188,히즈,"rural area, second chance, lgbt, gay theme, in...",7.200,24


In [ ]:
tmdb_df.to_csv('/content/tmdb_movies.csv', encoding='utf-8-sig', index = False)

### 파일 병합

In [ ]:
# CSV 불러오기
df_movies = pd.read_csv("2024_movies.csv")
df_tmdb = pd.read_csv("tmdb_movies.csv")

# 영화명 기준 병합 (movieNm ↔ title)
merged_df = pd.merge(df_movies, df_tmdb, how='left', left_on='movieNm', right_on='title')

# title 중복 컬럼 제거
merged_df.drop(columns=['title'], inplace=True)

In [ ]:
merged_df

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url,keywords,vote_average,vote_count
0,20228796,하얼빈,HARBIN,113.0,2024.0,20241224,드라마,우민호,"현빈, 박정민, 조우진",https://image.tmdb.org/t/p/w500/oNA9eRaFCLlFKR...,"espionage, spy, independence movement, biograp...",7.733,15.0
1,20227570,울산의 별,Star of Ulsan,117.0,2022.0,20240124,드라마,정기혁,"김금순, 최우빈, 변중희",https://image.tmdb.org/t/p/w500/7EnBkQpZKORRi8...,NaN,NaN,NaN
2,20200142,소방관,FIREFIGHTERS,106.0,2024.0,20241204,드라마,곽경택,"주원, 곽도원, 유재명",https://image.tmdb.org/t/p/w500/8claugsQFr5SAx...,"based on true story, firefighter",7.750,8.0
3,20234686,그 여름날의 거짓말,That Summer′s Lie,138.0,2024.0,20240828,"멜로/로맨스, 미스터리",손현록,"박서윤, 최민재, 유병용",https://image.tmdb.org/t/p/w500/7TiQBVFFVsjLxh...,NaN,NaN,NaN
4,20204271,행복의 나라,Land of Happiness,124.0,2024.0,20240814,드라마,추창민,"조정석, 이선균, 유재명",https://image.tmdb.org/t/p/w500/c5xAVbSdJYNVjA...,"court, 1970s, lawyer, korea president",6.800,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,20236759,소녀는 졸업하지 않는다,"Sayonara, Girls",120.0,2023.0,20240124,멜로/로맨스,나카가와 슌,"카와이 유미, 오노 리나, 코미야마 리나",https://image.tmdb.org/t/p/w500/k41x8dE8spurWv...,"high school, farewell, japan, based on novel o...",9.300,7.0
202,20235854,덤 머니,Dumb Money,105.0,2023.0,20240117,"코미디, 드라마",크레이그 질레스피,"폴 다노, 세스 로건, 아메리카 페레라",https://image.tmdb.org/t/p/w500/e9u7luSxFKOZgP...,"massachusetts, manipulation of the media, base...",6.700,641.0
203,20235073,라이즈,Rise,117.0,2022.0,20240117,"드라마, 코미디",세드릭 클라피쉬,"마리옹 바르보, 메디 바키, 호페쉬 쉑터",https://image.tmdb.org/t/p/w500/slkVQhehnP3h40...,"sports, biography, basketball, national basket...",7.607,276.0
204,20209574,히즈,his,126.0,2020.0,20240110,드라마,이마이즈미 리키야,"미야자와 히오, 후지와라 키세츠",https://image.tmdb.org/t/p/w500/ePBDOSLQ26voWs...,"rural area, second chance, lgbt, gay theme, in...",7.200,24.0


In [ ]:
merged_df.sort_values(by = 'openDt', ascending = False)

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url,keywords,vote_average,vote_count
6,20190300,보고타: 마지막 기회의 땅,Bogota: City of the Lost,106.0,2024.0,20241231,"범죄, 드라마",김성제,"송중기, 이희준, 권해효",https://image.tmdb.org/t/p/w500/p1WPGGPVHHVJwT...,"bogotá, 1990s, colombia, bogota, colombia, sus...",6.982,85.0
17,20244850,시빌 워: 분열의 시대,Civil War,109.0,2024.0,20241231,"액션, 전쟁, 드라마",알렉스 가랜드,"커스틴 던스트, 케일리 스패니, 와그너 모라",https://image.tmdb.org/t/p/w500/sh7Rg8Er3tFcN9...,"sniper, new york city, race against time, wash...",6.865,3560.0
12,20246777,더 폴: 디렉터스 컷,The Fall,119.0,2024.0,20241225,"드라마, 판타지",타셈 싱,"리 페이스, 카틴카 언타루",https://image.tmdb.org/t/p/w500/g3RKh7Gp2lDUnX...,"suicide attempt, stuntman, remake, morphine, h...",7.598,1366.0
0,20228796,하얼빈,HARBIN,113.0,2024.0,20241224,드라마,우민호,"현빈, 박정민, 조우진",https://image.tmdb.org/t/p/w500/oNA9eRaFCLlFKR...,"espionage, spy, independence movement, biograp...",7.733,15.0
15,20245977,극장판 짱구는 못말려: 우리들의 공룡일기,Crayon Shinchan the Movie: Our Dinosaur Diary,105.0,2024.0,20241218,애니메이션,사사키 시노부,"박영남, 강희선, 김환진",https://image.tmdb.org/t/p/w500/n7YcOeqPOmvQ2P...,"kindergarten, dinosaur, based on manga, seinen...",5.700,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,20235253,외계+인 2부,Alienoid,122.0,2023.0,20240110,기타,최동훈,"류준열, 김태리, 김우빈",https://image.tmdb.org/t/p/w500/8QVDXDiOGHRcAD...,NaN,NaN,NaN
144,20235353,길위에 김대중,Kim Dae Jung on the road,125.0,2023.0,20240110,다큐멘터리,민환기,"김대중, 김도영, 김재록",https://image.tmdb.org/t/p/w500/cuzjFGN08EvZgP...,kim dae-jung,8.200,4.0
199,20236557,스즈메의 문단속: 다녀왔어,Suzume,121.0,2022.0,20240110,애니메이션,신카이 마코토,"하라 나노카, 마츠무라 호쿠토",https://image.tmdb.org/t/p/w500/oNbCAM3UVJamxR...,NaN,NaN,NaN
204,20209574,히즈,his,126.0,2020.0,20240110,드라마,이마이즈미 리키야,"미야자와 히오, 후지와라 키세츠",https://image.tmdb.org/t/p/w500/ePBDOSLQ26voWs...,"rural area, second chance, lgbt, gay theme, in...",7.200,24.0


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieCd       206 non-null    int64  
 1   movieNm       206 non-null    object 
 2   movieNmEn     206 non-null    object 
 3   showTm        206 non-null    float64
 4   prdtYear      206 non-null    float64
 5   openDt        206 non-null    int64  
 6   genres        206 non-null    object 
 7   directors     206 non-null    object 
 8   actors        206 non-null    object 
 9   poster_url    206 non-null    object 
 10  keywords      144 non-null    object 
 11  vote_average  144 non-null    float64
 12  vote_count    144 non-null    float64
dtypes: float64(4), int64(2), object(7)
memory usage: 21.1+ KB


In [ ]:
# keyword가 비어 있는 행 삭제
idx6 = merged_df[merged_df['keywords'].isnull()].index
idx6

Index([  1,   3,   8,   9,  11,  13,  21,  23,  25,  27,  28,  33,  38,  40,
        41,  42,  46,  54,  55,  57,  61,  62,  63,  65,  75,  79,  83,  84,
        88,  90, 100, 104, 106, 109, 112, 114, 122, 126, 128, 129, 134, 136,
       140, 141, 143, 148, 159, 160, 161, 163, 164, 174, 179, 181, 182, 184,
       186, 191, 192, 193, 198, 199],
      dtype='int64')

In [ ]:
merged_df.drop(idx6, inplace = True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, 0 to 205
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieCd       144 non-null    int64  
 1   movieNm       144 non-null    object 
 2   movieNmEn     144 non-null    object 
 3   showTm        144 non-null    float64
 4   prdtYear      144 non-null    float64
 5   openDt        144 non-null    int64  
 6   genres        144 non-null    object 
 7   directors     144 non-null    object 
 8   actors        144 non-null    object 
 9   poster_url    144 non-null    object 
 10  keywords      144 non-null    object 
 11  vote_average  144 non-null    float64
 12  vote_count    144 non-null    float64
dtypes: float64(4), int64(2), object(7)
memory usage: 15.8+ KB


In [ ]:
merged_df.head(10)

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url,keywords,vote_average,vote_count
0,20228796,하얼빈,HARBIN,113.0,2024.0,20241224,드라마,우민호,"현빈, 박정민, 조우진",https://image.tmdb.org/t/p/w500/oNA9eRaFCLlFKR...,"espionage, spy, independence movement, biograp...",7.733,15.0
2,20200142,소방관,FIREFIGHTERS,106.0,2024.0,20241204,드라마,곽경택,"주원, 곽도원, 유재명",https://image.tmdb.org/t/p/w500/8claugsQFr5SAx...,"based on true story, firefighter",7.750,8.0
4,20204271,행복의 나라,Land of Happiness,124.0,2024.0,20240814,드라마,추창민,"조정석, 이선균, 유재명",https://image.tmdb.org/t/p/w500/c5xAVbSdJYNVjA...,"court, 1970s, lawyer, korea president",6.800,10.0
5,20234713,딸에 대하여,Concerning My Daughter,105.0,2023.0,20240904,드라마,이미랑,"오민애, 허진, 임세미",https://image.tmdb.org/t/p/w500/jmcVE6CRv6I3cz...,"based on novel or book, mother daughter relati...",6.500,3.0
6,20190300,보고타: 마지막 기회의 땅,Bogota: City of the Lost,106.0,2024.0,20241231,"범죄, 드라마",김성제,"송중기, 이희준, 권해효",https://image.tmdb.org/t/p/w500/p1WPGGPVHHVJwT...,"bogotá, 1990s, colombia, bogota, colombia, sus...",6.982,85.0
7,20233217,더 킬러스,The Killers,118.0,2023.0,20241023,"드라마, 액션, 스릴러","이명세, 장항준, 노덕, 김종관","심은경, 연우진, 홍사빈",https://image.tmdb.org/t/p/w500/dB6Krk806zeqd0...,"husband wife relationship, based on novel or b...",7.428,3607.0
10,20248530,아침바다 갈매기는,The Land of Morning Calm,113.0,2024.0,20241127,"드라마, 가족",박이웅,"윤주상, 양희경, 박종환",https://image.tmdb.org/t/p/w500/lnMJXfMHfUIhGV...,"fisherman, death of family",6.167,3.0
12,20246777,더 폴: 디렉터스 컷,The Fall,119.0,2024.0,20241225,"드라마, 판타지",타셈 싱,"리 페이스, 카틴카 언타루",https://image.tmdb.org/t/p/w500/g3RKh7Gp2lDUnX...,"suicide attempt, stuntman, remake, morphine, h...",7.598,1366.0
14,20233261,청설,Hear Me: Our Summer,108.0,2024.0,20241106,"드라마, 멜로/로맨스",조선호,"홍경, 노윤서, 김민주",https://image.tmdb.org/t/p/w500/i4VmtHRy4xRkxI...,remake,7.500,13.0
15,20245977,극장판 짱구는 못말려: 우리들의 공룡일기,Crayon Shinchan the Movie: Our Dinosaur Diary,105.0,2024.0,20241218,애니메이션,사사키 시노부,"박영남, 강희선, 김환진",https://image.tmdb.org/t/p/w500/n7YcOeqPOmvQ2P...,"kindergarten, dinosaur, based on manga, seinen...",5.700,30.0


In [ ]:
merged_df.to_csv('/content/tmdb144_movies.csv',  encoding='utf-8-sig', index = False)

In [ ]:
movies3_df = pd.read_csv('/content/tmdb144_movies.csv')
movies3_df

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url,keywords,vote_average,vote_count
0,20228796,하얼빈,HARBIN,113.0,2024.0,20241224,드라마,우민호,"현빈, 박정민, 조우진",https://image.tmdb.org/t/p/w500/oNA9eRaFCLlFKR...,"espionage, spy, independence movement, biograp...",7.733,15.0
1,20200142,소방관,FIREFIGHTERS,106.0,2024.0,20241204,드라마,곽경택,"주원, 곽도원, 유재명",https://image.tmdb.org/t/p/w500/8claugsQFr5SAx...,"based on true story, firefighter",7.750,8.0
2,20204271,행복의 나라,Land of Happiness,124.0,2024.0,20240814,드라마,추창민,"조정석, 이선균, 유재명",https://image.tmdb.org/t/p/w500/c5xAVbSdJYNVjA...,"court, 1970s, lawyer, korea president",6.800,10.0
3,20234713,딸에 대하여,Concerning My Daughter,105.0,2023.0,20240904,드라마,이미랑,"오민애, 허진, 임세미",https://image.tmdb.org/t/p/w500/jmcVE6CRv6I3cz...,"based on novel or book, mother daughter relati...",6.500,3.0
4,20190300,보고타: 마지막 기회의 땅,Bogota: City of the Lost,106.0,2024.0,20241231,"범죄, 드라마",김성제,"송중기, 이희준, 권해효",https://image.tmdb.org/t/p/w500/p1WPGGPVHHVJwT...,"bogotá, 1990s, colombia, bogota, colombia, sus...",6.982,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,20236759,소녀는 졸업하지 않는다,"Sayonara, Girls",120.0,2023.0,20240124,멜로/로맨스,나카가와 슌,"카와이 유미, 오노 리나, 코미야마 리나",https://image.tmdb.org/t/p/w500/k41x8dE8spurWv...,"high school, farewell, japan, based on novel o...",9.300,7.0
140,20235854,덤 머니,Dumb Money,105.0,2023.0,20240117,"코미디, 드라마",크레이그 질레스피,"폴 다노, 세스 로건, 아메리카 페레라",https://image.tmdb.org/t/p/w500/e9u7luSxFKOZgP...,"massachusetts, manipulation of the media, base...",6.700,641.0
141,20235073,라이즈,Rise,117.0,2022.0,20240117,"드라마, 코미디",세드릭 클라피쉬,"마리옹 바르보, 메디 바키, 호페쉬 쉑터",https://image.tmdb.org/t/p/w500/slkVQhehnP3h40...,"sports, biography, basketball, national basket...",7.607,276.0
142,20209574,히즈,his,126.0,2020.0,20240110,드라마,이마이즈미 리키야,"미야자와 히오, 후지와라 키세츠",https://image.tmdb.org/t/p/w500/ePBDOSLQ26voWs...,"rural area, second chance, lgbt, gay theme, in...",7.200,24.0


In [ ]:
# vote_count 100개 이하인 행 삭제
movies3_df = movies3_df[movies3_df['vote_count'] > 100]
movies3_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, 5 to 141
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieCd       79 non-null     int64  
 1   movieNm       79 non-null     object 
 2   movieNmEn     79 non-null     object 
 3   showTm        79 non-null     float64
 4   prdtYear      79 non-null     float64
 5   openDt        79 non-null     int64  
 6   genres        79 non-null     object 
 7   directors     79 non-null     object 
 8   actors        79 non-null     object 
 9   poster_url    79 non-null     object 
 10  keywords      79 non-null     object 
 11  vote_average  79 non-null     float64
 12  vote_count    79 non-null     float64
dtypes: float64(4), int64(2), object(7)
memory usage: 8.6+ KB


In [ ]:
movies3_df.to_csv('/content/tmdb79_movies.csv', encoding = 'utf-8-sig', index = False)